In [1]:
import glob
import sys
import numpy as np

from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

import dpdm_delay

# import customized files to introduce 2T modes.
import gp_bases_new as gp_bases
import pl_prior_new as gp_priors

from enterprise.signals import selections

import multiprocess
from PTMCMCSampler.PTMCMCSampler import PTSampler


Cannot import PINT? Meh...


In [2]:
datadir = 'DR2'

parfiles = sorted(glob.glob(datadir + '/newpars/*.par'))
timfiles = sorted(glob.glob(datadir + '/toas/*.tim'))

In [3]:
psrs=[]
for ipsr in range(len(parfiles)):
    psr = Pulsar(parfiles[ipsr], timfiles[ipsr],ephem="DE436",clk="TT(BIPM2018)")
    psrs.append(psr)
    print(psr.name)

J0437-4715
J0613-0200
J0711-6830


J1017-7156
J1022+1001
J1024-0719
J1045-4509


J1125-6014


J1446-4701


J1545-4550
J1600-3053
J1603-7202
J1643-1224
J1713+0747
J1730-2304
J1732-5049
J1744-1134
J1824-2452A


J1832-0836
J1857+0943
J1909-3744
J1939+2134
J2124-3358
J2129-5721
J2145-0750


J2241-5236


In [4]:
# red noise
nmodes = 30


log10_A = parameter.Constant()
gamma = parameter.Constant()
pl = gp_priors.powerlaw(log10_A=log10_A, gamma=gamma)
dm_basis = gp_bases.createfourierdesignmatrix_dm(nmodes=nmodes)
red_basis = gp_bases.createfourierdesignmatrix_red(nmodes=nmodes)
selection = selections.Selection(selections.by_band)

dmn = gp_signals.BasisGP(pl, dm_basis, name='dm', coefficients=False)
spn = gp_signals.BasisGP(pl, red_basis, name='sp',coefficients=False)
bdn = gp_signals.BasisGP(pl, red_basis, name='bd',coefficients=False,selection=selection)

# white noise
backend = selections.Selection(selections.by_backend)
efac = parameter.Constant()
equad = parameter.Constant()

ef = white_signals.MeasurementNoise(efac=efac,selection=backend)
eq = white_signals.EquadNoise(log10_equad=equad,selection=backend)
wnb = ef + eq

# timing model
tm = gp_signals.TimingModel()

# Bayes ephmeris
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)


# include best fit parameters
Dict = np.load('noise_params/noisepars_m'+str(nmodes)+'.npy',allow_pickle=True,encoding='bytes').item()
len(Dict)

638

## Prepare for Bayes Limits

In [5]:
def single_task(log10_ma, name, ret, model, test=False):
    
        np.random.seed()
    
        log10_eps = parameter.Uniform(-28.0, -16.0)('x_dp_log10_eps')
        dp = dpdm_delay.dpdm_block(model=model,log10_ma=log10_ma,log10_eps=log10_eps)

        model0  = tm + wnb + dmn + spn + eph + dp
        model1  = tm + wnb + dmn + spn + eph + dp + bdn

        signal=[]
        for psr in psrs:
            if psr.name in ['J0437-4715','J1939+2134']:
                signal.append( model1( psr ) )
            else:
                signal.append( model0( psr ) )

        pta = signal_base.PTA(signal)
        pta.set_default_params(Dict)
        if test == True:
            return pta

        #outDir="bayes/"+model+"_m"+str(nmodes)+"/n"+str(i).zfill(3)+'/'
        outDir="bayes/"+model+"_m"+str(nmodes)+'/'+name+'/'
#============================
        # to make the initial values
        xs = {par.name:par.sample() for par in pta.params}
        for parname in Dict.keys():
            if parname in xs.keys():
                xs[parname] = Dict[parname]
        
        x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
        ndim = len(x0)
#============================

        x0[-19:-8] = 0.
        x0[-8] = -26.
        
        groups=[range(ndim),range(ndim-8,ndim)]
        
        print(pta.get_lnlikelihood(x0))
        
        
        covdiag = np.ones(ndim)*0.01

        sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior,
                        cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

        sampler.sample(x0, 400000, isave=1000)
        
        #chain=np.loadtxt(outDir+'/chain_1.txt')

        #logep=chain[int(0.8*len(chain)):,-12]

        #ret.value = np.percentile(logep,95)


## Run the code

In [6]:
name = 'A'
ncores = 6
logma_grid = np.linspace(-23.5,-21.5,126)

for j in range(125//6+1):
    a = ncores*j
    b = min(ncores*(j+1),125)

    print(a,b,nmodes)

    jobs = []
    RETs={}

    for i in range(a,b): 

        RETs[i] = multiprocess.Manager().Value('i',0)

        logmamin = logma_grid[a]
        logmamax = logma_grid[b]

        log10_ma = parameter.Uniform(logmamin,logmamax)('x_dp_log10_ma')#n=0~99

        p = multiprocess.Process(target=single_task, args=(log10_ma, name+str(i).zfill(3) , RETs[i] , "U1BL" ))

        jobs.append(p)

        p.start()


    for p in jobs:

        p.join()
    


0 6 30
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L


INFO: enterprise.signals.signal_base: Setting J0437-4715_CASPSR_40CM_efac to 1.1821247662122054
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_10CM_efac to 1.0975825129925123
INFO: enterprise.signals.signal_base: Setting J0437-4715_CASPSR_40CM_efac to 1.1821247662122054
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_20CM_efac to 1.0248557925516382
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_10CM_efac to 1.0975825129925123
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_20CM_efac to 1.0248557925516382
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_50CM_efac to 1.0204302047551015
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_10CM_efac to 1.010201435266573
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_1433_efac to 0.22891987956193693
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_20CM_efac to 0.9537635425047156
INFO: enterprise.signals.signal_base: Setting J0

INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_1433_efac to 1.020031904922391
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_20CM_efac to 0.9834454388918584
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_20CM_efac to 0.9834454388918584
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_early_10CM_efac to 0.9063247997889078
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_early_10CM_efac to 0.9063247997889078
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_early_20CM_efac to 1.0254002422973625
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_early_20CM_efac to 1.0254002422973625
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB_10CM_efac to 1.003489904341146
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB_20CM_efac to 1.0081747545083555
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB_10CM_efac to 1.003489904341146
INFO: enterprise.signals.signal_

INFO: enterprise.signals.signal_base: Setting J0711-6830_dm_gamma to 6.093837176841098
INFO: enterprise.signals.signal_base: Setting J0711-6830_sp_log10_A to -12.862481000431677
INFO: enterprise.signals.signal_base: Setting J0711-6830_CASPSR_40CM_log10_equad to -7.663005915869408
INFO: enterprise.signals.signal_base: Setting J0711-6830_CPSR2_20CM_log10_equad to -5.940468437823865
INFO: enterprise.signals.signal_base: Setting J0711-6830_sp_gamma to 0.6265684246369604
INFO: enterprise.signals.signal_base: Setting J0711-6830_CPSR2_50CM_log10_equad to -7.426871787030082
INFO: enterprise.signals.signal_base: Setting J1017-7156_CASPSR_40CM_efac to 0.8588849502216462
INFO: enterprise.signals.signal_base: Setting J0711-6830_PDFB1_10CM_log10_equad to -5.118848201382466
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_10CM_efac to 0.9796248949731852
INFO: enterprise.signals.signal_base: Setting J0711-6830_PDFB1_1433_log10_equad to -6.5940778805189
INFO: enterprise.signals.signal_bas

INFO: enterprise.signals.signal_base: Setting J1022+1001_dm_log10_A to -13.02930208262561
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB1_early_10CM_log10_equad to -6.069753621618791
INFO: enterprise.signals.signal_base: Setting J1022+1001_dm_gamma to 0.10461941266211981
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB1_early_20CM_log10_equad to -5.933270467615061
INFO: enterprise.signals.signal_base: Setting J1022+1001_sp_log10_A to -12.779024552610808
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_10CM_log10_equad to -5.622084700758624
INFO: enterprise.signals.signal_base: Setting J1022+1001_sp_gamma to 0.3130480260190169
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_20CM_log10_equad to -6.066525990131196
INFO: enterprise.signals.signal_base: Setting J1024-0719_CASPSR_40CM_efac to 1.0484249315138279
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_40CM_log10_equad to -8.173915335923978
INFO: enterprise.signals.si

...

-inf


/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/parameter.py:66: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/usr/local/lib/python3.6/dist-packages/PTMCMCSampler-2015.2-py3.6.egg/PTMCMCSampler/PTMCMCSampler.py:494: RuntimeWarning: invalid value encountered in double_scalars
  diff = newlnprob - lnprob0 + qxy
/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/parameter.py:66: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
Process Process-6:
  File "/usr/local/lib/python3.6/dist-packages/PTMCMCSampler-2015.2-py3.6.egg/PTMCMCSampler/PTMCMCSampler.py", line 490, in PTMCMCOneStep
    newlnlike = self.logl(y)
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-4:
  File "/home/xuex/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
   

KeyboardInterrupt: 

  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 576, in <listcomp>
    signalcollection in self._signalcollections]
  File "/usr/local/lib/python3.6/dist-packages/PTMCMCSampler-2015.2-py3.6.egg/PTMCMCSampler/PTMCMCSampler.py", line 362, in sample
    p0, lnlike0, lnprob0, iter)
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 871, in get_phi
    phi = phi.add(signal.get_phi(params), self._idx[signal])
Process Process-10:
Process Process-8:
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/gp_signals.py", line 192, in get_phi
    for key, slc in self._slices.items():
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 176, in __call__
    TNrs = self.pta.get_TNr(params)
  File "/usr/local/lib/python3.6/dist-packages/PTMCMCSampler-2015.2-py3.6.egg/PTMCM

  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 274, in get_TNr
    in self._signalcollections]
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 354, in get_phiinv
    return self.get_phiinv_byfreq_cliques(params, logdet)
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/parameter.py", line 483, in __call__
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 273, in <listcomp>
    return [signalcollection.get_TNr(params) for signalcollection
  File "<__array_function__ internals>", line 6, in dot
  File "/usr/local/lib/python3.6/dist-packages/enterprise-1.2.1-py3.6.egg/enterprise/signals/signal_base.py", line 463, in get_phiinv_byfreq_cliques
    phi = self.get_phi(params, cliques=True)
  File "/usr/local/lib/python3.6